In [32]:
#Script to evaluate the determinants of patent citation 
#Renato Kogeyama


#Miami, December 24th, 2019
# Prof. Rafael Corredoira suggested:
# - Inclusion of a tree of citations
#   To track back the source of citations. This is information is not given by direct count of citations.
# - Consider policy changes in the way patents are cited
#   Policy changes in 2000 changed the time frame of citation, and 2010 partially moved citation to applications
# - Track classification changes 
#   The original classification system in USPTO changed from a technical based to a market based classification system
#   See if there is an impact
# - Consider a text analysis of the claims
#   Classification is based on the claims but it is not clear how many claims are related to each classification category
# - Include moderation effect from classification
#   Citations patterns may change across industries, so some effects may disappear if industry is not accounted for.

# In summary, his ideas help increase structure of the current work


#Syracuse, December 3rd, 2019

#The original script is getting too complex
#There was many tentative scripts to play with data
#Here I am writing a script to show the relevance of variables to patent citation

#11-12-2019
#Introducing normalization

#10-11-2019
#I introduced log backward citation, what corrects for very dispersed results
#but the major problem is that few patents receive citations
#bring back binary output

#10-10-2019
#Added graphics and new distributions

#10-03-2019
#I rewrote the citation data to clean the strings

#09-15-2019
#O naive bayes tem algum problema com distribuicoes desbalanceadas
#o scikit learn tem um modulo que corrige count distributions com muitos zeros, o complementNB
#porem este nao esta disponivel na atual versao disponibilizada no HPC da FIU

#09-10-2019
#o trabalho pede uma abordagem mais sistematica e cuidadosa
#estou agrupando o codigo antigo comentado e vou comecar um novo codigo

#09-27-2019
#I am renaming citation as forward citation and backward citation

#09-17-2018

#Alto uso de memoria - rodar no Amazon AWS 



In [33]:
import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn import linear_model, datasets
from sklearn.model_selection import train_test_split

from scipy import stats
import datetime
import matplotlib.pyplot as plot
import pylab as pl

from math import sqrt


import sys
sys.path.append('/home/rkogeyam/scripts/')

from best_num_attr import best_num_attr
from xattrSelect import xattrSelect
from sampler import sampler
from normalize import normalize


In [34]:
%matplotlib inline

In [35]:
fname='/home/rkogeyam/PATENT_CITATION/df_w_pat_cit_191003.csv'

df=pd.read_csv(fname)

# sample_size=1000
# df=sampler(fname, sample_size)

/home/applications/anaconda3/4.2.0/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1,3,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [45]:
df=df.drop('date', axis=1)

df=df.dropna()

In [36]:
df=df.iloc[:,[1,2,5,8,9,11,12,13,14,15]]

In [37]:
df=df.set_index('id')

In [46]:
# defensive publication and statutory registration are not wrong entries
# I will just rename them and leave them in the dataset
df.groupby('type').count()

,kind,num_claims,forward_citation,backward_citation,year,month,day
type,,,,,,,
TVPP,3,3,3,3,3,3,3
defensive publication,509,509,509,509,509,509,509
design,563003,563003,563003,563003,563003,563003,563003
plant,24790,24790,24790,24790,24790,24790,24790
reissue,17812,17812,17812,17812,17812,17812,17812
statutory invention registration,2254,2254,2254,2254,2254,2254,2254
utility,5879879,5879879,5879879,5879879,5879879,5879879,5879879


In [48]:
df.rename(columns={'defensive publication': 'defensive', 'statutory invention registration': 'statutory'}, inplace=True)

In [40]:
# # df.dtypes
# df=df.drop('date', axis=1)

# df=df.dropna()

# df.describe().transpose()

# # #normalization
# df=normalize(df)

# df.describe().transpose()

# # List of IVs
# chosenColumns=df.columns.values.tolist()
# len(chosenColumns)

# chosenColumns.remove('backward_citation')
# len(chosenColumns)

# # chosenColumns.remove('citation_bi')
# # chosenColumns.remove('log_back_cit')

# myX = df.as_matrix(columns=chosenColumns)
# myY = df.as_matrix(columns=['backward_citation'])

# xTrain, xTest, yTrain, yTest = train_test_split(myX, myY, train_size=0.7, random_state=3) 
# testSize = yTest.shape[0]
# trainSize = yTrain.shape[0]

# best_num_attr(myX, xTrain, xTest, yTrain, yTest, chosenColumns, regtype='linear')